In [2]:
import matplotlib.pyplot as plt

In [3]:
from collections import namedtuple
import pandas as pd
#pd.style.set_properties(subset=['ad_description'], **{'width-min': '300px'})


fnames = []
new_fnames = []
KW = ['D', 'X', 'R', 'S', 'M', 'N', 'A', 'B']
Key = namedtuple('Key', KW + ['i'])
keys = []
datasets = ['cifarfs', 'fc100', 'miniimagenet']# 'miniimagenet84',]# 'tieredimagenet']
for i in [1, 2, 3]:
    for D in datasets:
        for M in ['']:
            for N in [0.2, 0.4, 0.6]:
                for NT in [N]:
                    for X in [0]:
                        for S in ['00.0']:
                            for R in [0.0, ]:
                                for A in [1, 10, 100]:
                                    for B in [-1, 0.5, 1, 10]:
                                        if B == -1 and A > 1:
                                            continue
                                        if B == -1:
                                            fnames.append(f'features.{D}{M}_{R}_n{N}_x{X}_s{S}_#{i}.pt55.lam1000.0_nt{NT}.csv')     
                                        else:
                                            fnames.append(f'features.{D}{M}_{R}_n{N}_x{X}_s{S}_a{A}_b{B}_#{i}.pt55.lam1000.0_nt{NT}.csv')
                                        keys.append(Key(D=D, X=X, R=R, S=float(S), M=M, N=N, A=A, B=B, i=i))
                                

In [4]:
recs = []
shots = [5]
mets = [
'test_acc', 
# 'test_acc_me1', 
# 'test_acc_me2',
# 'test_acc_me3',
# 'test_acc_me4',
# 'test_acc_me5',
# 'test_acc_me6',
# 'test_acc_me7',
# 'test_acc_me8',
# 'test_acc_me9',
'test_acc_med',
'test_acc_maj',

]

metrics = set()
for key, fname in zip(keys, fnames):
    #print(key)
    rec = {}
    cdf = pd.read_csv(f'results/{fname}')
    #print(cdf.columns)
    #rec['test_acc_80'] = cdf[cdf.shots==80].test_acc*100
    for m in mets:
        for s in shots:
            metric = f'{m}_{s}' 
            rec[metric] = float(cdf[m][cdf.shots==s]*100)
            metrics.add(metric)  
    for k in KW +['i']:
        rec[k] = key.__getattribute__(k)
    recs.append(rec)
metrics = sorted(list(metrics))
#metrics = ['test_acc_5', 'test_acc_80', 'test_acc_lin_5', 'test_acc_lin_80']






In [5]:

df = pd.DataFrame(recs)
print(KW)
df = df.groupby(KW).mean().reset_index()

print(df.to_markdown())
print(df.groupby('N').mean())

['D', 'X', 'R', 'S', 'M', 'N', 'A', 'B']
|    | D            |   X |   R |   S | M   |   N |   A |    B |   test_acc_5 |   test_acc_med_5 |   test_acc_maj_5 |   i |
|---:|:-------------|----:|----:|----:|:----|----:|----:|-----:|-------------:|-----------------:|-----------------:|----:|
|  0 | cifarfs      |   0 |   0 |   0 |     | 0.2 |   1 | -1   |      70.528  |          70.6911 |          70.4818 |   2 |
|  1 | cifarfs      |   0 |   0 |   0 |     | 0.2 |   1 |  0.5 |      72.6449 |          72.8733 |          72.5676 |   2 |
|  2 | cifarfs      |   0 |   0 |   0 |     | 0.2 |   1 |  1   |      72.2791 |          72.4342 |          72.1773 |   2 |
|  3 | cifarfs      |   0 |   0 |   0 |     | 0.2 |   1 | 10   |      64.9005 |          65.0556 |          64.9525 |   2 |
|  4 | cifarfs      |   0 |   0 |   0 |     | 0.2 |  10 |  0.5 |      72.3253 |          72.5409 |          72.24   |   2 |
|  5 | cifarfs      |   0 |   0 |   0 |     | 0.2 |  10 |  1   |      72.0609 |          72

In [15]:
for D in df.D.unique():
    for N in df.N.unique():
        for metric in ['test_acc_5']:
            print(f'dataset: {D}\t metric: {metric}\tnoise: {N}')
            pt = df[df.N == N].pivot_table(values = metric, index='A', columns='B')
            pt = pt.fillna(0)
            print(pt)
            amax = pt.to_numpy().argmax()
            print(pt.to_numpy().max())
            row, col = (amax//pt.shape[1], amax%pt.shape[1])
            print(f'(A,B)={(pt.index[row], pt.columns[col])}')
            print()

dataset: cifarfs	 metric: test_acc_5	noise: 0.2
B       -1.0        0.5        1.0        10.0
A                                             
1    63.23408  65.285933  65.174526  58.998524
10    0.00000  65.067265  64.995562  59.134228
100   0.00000  64.442970  64.485488  59.038080
65.2859331111111
(A,B)=(1, 0.5)

dataset: cifarfs	 metric: test_acc_5	noise: 0.4
B        -1.0        0.5        1.0        10.0
A                                              
1    50.540447  52.268299  52.192448  48.151114
10    0.000000  51.991412  52.520744  47.810817
100   0.000000  51.996594  52.258818  48.296596
52.52074444444444
(A,B)=(10, 1.0)

dataset: cifarfs	 metric: test_acc_5	noise: 0.6
B        -1.0        0.5        1.0        10.0
A                                              
1    35.168003  36.089485  36.245634  33.987409
10    0.000000  35.889633  36.240596  33.725782
100   0.000000  35.974967  36.143854  34.052004
36.24563366666667
(A,B)=(1, 1.0)

dataset: fc100	 metric: test_acc_5	nois

In [ ]:
for D in df.D.unique():
    for N in df.N.unique():
        print(f'dataset: {D}\tnoise: {N}')
        print(df[df.N == N][df.A == 1][['A', 'B'] + metrics])
        print()

dataset: cifarfs	noise: 0.2
    A     B  test_acc_5  test_acc_maj_5  test_acc_med_5
0   1  -1.0   70.528009       70.481789       70.691119
1   1   0.5   72.644899       72.567563       72.873343
2   1   1.0   72.279123       72.177348       72.434237
3   1  10.0   64.900451       64.952451       65.055562
30  1  -1.0   50.662222       50.721332       50.801780
31  1   0.5   52.120448       52.081778       52.290671
32  1   1.0   52.377332       52.458667       52.558672
33  1  10.0   47.046227       47.128001       47.054670
60  1  -1.0   68.512009       68.444012       68.959564
61  1   0.5   71.092452       70.987124       71.559567
62  1   1.0   70.867121       70.679120       71.303562
63  1  10.0   65.048896       64.935118       65.372007

dataset: cifarfs	noise: 0.4
    A     B  test_acc_5  test_acc_maj_5  test_acc_med_5
10  1  -1.0   56.900447       56.756001       57.125788
11  1   0.5   58.632001       58.658668       58.812896
12  1   1.0   57.803110       57.807559       5

/tmp/ipykernel_2525787/2329106049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.N == N][df.A == 1][['A', 'B'] + metrics])
/tmp/ipykernel_2525787/2329106049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.N == N][df.A == 1][['A', 'B'] + metrics])
/tmp/ipykernel_2525787/2329106049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.N == N][df.A == 1][['A', 'B'] + metrics])
/tmp/ipykernel_2525787/2329106049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.N == N][df.A == 1][['A', 'B'] + metrics])
/tmp/ipykernel_2525787/2329106049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.N == N][df.A == 1][['A', 'B'] + metrics])
/tmp/ipykernel_2525787/2329106049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.N == N][df

    A     B  test_acc_5  test_acc_maj_5  test_acc_med_5
20  1  -1.0   38.706227       38.916004       38.624892
21  1   0.5   39.700449       39.922672       39.475114
22  1   1.0   39.911116       40.192448       39.731558
23  1  10.0   36.235558       36.629780       36.077782
50  1  -1.0   29.958224       30.065336       29.964002
51  1   0.5   30.713779       30.712448       30.658669
52  1   1.0   30.484892       30.540891       30.412446
53  1  10.0   28.785335       29.011999       28.721335
80  1  -1.0   36.839558       37.145780       36.892892
81  1   0.5   37.854227       38.149781       37.728003
82  1   1.0   38.340893       38.556002       38.229782
83  1  10.0   36.941335       37.236450       36.890226

dataset: miniimagenet	noise: 0.2
    A     B  test_acc_5  test_acc_maj_5  test_acc_med_5
0   1  -1.0   70.528009       70.481789       70.691119
1   1   0.5   72.644899       72.567563       72.873343
2   1   1.0   72.279123       72.177348       72.434237
3   1  10.0   

/tmp/ipykernel_2525787/2329106049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.N == N][df.A == 1][['A', 'B'] + metrics])
/tmp/ipykernel_2525787/2329106049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.N == N][df.A == 1][['A', 'B'] + metrics])
/tmp/ipykernel_2525787/2329106049.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.N == N][df.A == 1][['A', 'B'] + metrics])
